
# Introduction to the CDS Data Engineering Team

## Our mission statement:
_"To expand the computational capabilities of Cornell Data Science in tackling industrial scale of data volume, speed and variety."_

## What is Data Engineering?
Data engineering usually refers to a broad field of process optimization for data science operations. It includes, but is not limited to, optimizations of resource allocation in data science operations, effective warehousing of large volumes of data, and efficiency of computations while maintaining integrity of the system. Note that our use of the term ‘resource’ refers to both physical things like CPUs, disk storage, network I/O and also intangible resources such as time and flexibility. Often these optimizations occur on a hardware level, where we adjust system settings on servers to accomodate for the diverse needs of different data tasks.

## What does the CDS Data Engineering Team do?
In the context of Cornell Data Science, we have a x-node cluster, that is to say, we have a system of x servers, with a total of x CPU cores, x amounts of RAM, and x TB of hard disk. We also have a GPU specifically for our deep learning needs. Our job is to understand how these work, how best to utilize them, identify good practices and protocols to stick to, and constantly explore ways to expand its capabilities. We will often say that our purpose is to “break the servers.” This is because only then can we understand what works/ doesn’t work, and what to do better.


## High-level take on performance enhancement
Speed is often the single most important factor in data engineering. With the current in extremely large and fast-paced data volumes, even the smallest inefficiencies can blow up to insurmountable problems in machine learning. Here we list certain ways we try to achieve faster speed in machines:

### 1. Parallelize operations

All computations can be broken down to constituent sub-operations that fall into these two categories: __Embarrassingly Parallel__ vs __Inherently sequential__. The first term refers to processes that can, given enough resources, be completed in one step in parallel, while the second term refers to processes that must follow non-concurrent steps to complete its objective. Parallelizing computations is an essential part of any industrial-scale data task and can have huge benefits. Let us see how this can be done.

Take, for example, a rather trivial process of creating a copy of an array stored in a certain location.(of course, the whole point of the example is to show that the trivial actually turns out to be the non-trivial!) The first element of the existing array is replicated as the first element of the new sequence, with all other elements following suit. This means that, if you have an original sequence of length n, all you need is n processors to transcribe one element each and complete the processes in one step. Therefore the transcription process can seem embarrassingly parallel.This is particularly useful if the said sequence is extremely large and the copy is urgently needed. If resources are available, the transcription can complete several magnitudes faster than what would take for a single processing unit could handle.

<br>

![Paralellism](https://computing.llnl.gov/tutorials/parallel_comp/images/parallelProblem.gif)    

<br>

However, there are caveats to this nifty trick in reality: a machine, even with infinite resources, will not be able to complete the transcription in just one step. For instance, even before the transcription can take place, the machine must find an available storage location for the new sequence, determine the size of storage to allocate, and reserve such a space by creating a pointer. Moreover, these processes must be preceded by the task of determining which cpu gets which element of the array, which can be a complicated assignment task if we wish to coordinate an optimal schedule. In fact, we have also glossed over the complications that come from whether the original array already sits in RAM or in a hard disk. 
  
Therefore at a high level, copying an array from one place to another in storage can be vastly shortened in duration using parallelism, but cannot entirely avoid being inherently sequential. This is why parallelizing is only part of the solution for most data engineering problems.  
 
### 2. Remove redundant operations (Initialize and Vectorize)

We demonstrated from the toy example above that as a whole, copying an array to another location entails inherently sequential tasks. Therefore a good way to shorten the duration of the task can be to actually eliminate __redundant operations__, which would effectively reduce the total number of operations and consequently the total duration of the operation. Let's revisit the copying-an-array example again to see how this could possibly work, this time in code form:

```
original_array = [1,2,3,4,5,6,7,8]
copied_array = []
for element in array:
  copied_array.append(element)
```
The code looks rather simple, but has many avoidable redundancies lurking inside. To give a little background before we dive into these redundancies, it must be noted that a Python list is simply an array of pointers to different objects. Therefore elements of a list can be of diverse data structures and types, allowing substantial flexibility. However such flexibility comes at a cost: the python interpreter must constantly type-check when iterating or appending to a list to make sure that whatever operation is being done to the list is allowed and the appropriate action taken. For example, a '+' operation can be a numeric addition for a numeric type, but a concatenation when applied to strings. This flexibility in data structures and the implicit type-checking operations are parts of the reason why python is much, much slower compared to languages like C, C++ or Java. (keep in mind however that despite this Python is very widely used in data science for its interpretability and ease of editing)  
<br>
Therefore in the above example, even though our original sequence is in fact all numeric types, Python must constantly check the types of the elements of the original array before appending a copy of it to the new array. Moreover, at every iteration, the machine must locate and allocate new memory spaces. Such operations are clearly redundant, given that we already know before we create the copy the type and length of the sequence to be copied. A common way to tackle the redundant memory allocation problem is proper initialization, as shown below:


```
original_array = [1,2,3,4,5,6,7,8]
length_original = len(original_array)
copied_array = [] * length_original

for element_index in length_original:
  copied_array[element_index] = original_array[element_index]
```

Here, instead of initializing an empty list first and iteratively adding new elements, the new array is first _initialized_ as a list filled with null values with the same length as the original array. This memory allocation is done only once, therefore reducing the total duration of the operation. As for the type-checking problem, typically a way to avoid such prolbems is to use a specialized data structure called a _numpy array_. A numpy array coerces all its elements to a single type, allowing fast iterative operations that are type-specific. An example is shown below:

```
original_array = np.array([1,2,3,4,5,6,7,8])
copied_array = np.array(original_array)

# please note the np.array(original_array) makes sure that copied_array is indeed a pointer to a copy, and not to the original object
```
Reducing redundant type-checking and implicit interpreter operations this way is often referred to as _vectorization_ and is often one of the best ways to speed up data science operations in commonly used languages like Python and R. Initialization and vectorization are useful data engineering skills to have to speed up your executions.

<br>
For more information on numpy, please refer to the [numpy documentation](http://www.numpy.org/)


### 3. Making individual operations faster 


While this may sound like a trivial solution,(equivalent "How should I win a 100m sprint?" answered by "Make sure each step is fast") in data science operations often can be made faster either by choosing to solve a different problem that can approximate the true solution, or to choose a different type of machinery/platform to speed up operations. 

<br>
__Trying to approximate rather than solve directly__   



The above image shows the optimization setup for the support vector classifier, a popular classification algorithm. Not just SVMs, but amost all machine learning problems are optimization problems: certain assumption are incorporated into a loss/error function that must be minimized given some input information. Thus advances in this optimization techniques often lead to improvements in machine learning algorithms, as even a small improvement in efficiency can, in large scales, be the difference between the infeasible and the feasible. A detailed overview of optimization techniques is very much beyond the scope of this tutorial, and we would not be able to do the field justice even if we tried. However, we can say that a common theme in optimization is attempting to approximate the optimal solution by solving a different, but easier problem. This technique is commonly referred to as _relaxation_.

<br>
![Primal Problem for Support Vector Classifier](http://scikit-learn.sourceforge.net/0.5/_images/math/eb74783f01c85187766959706f842a74224e10f4.png =300x)

Solving relaxations, rather than directly solve for the optimal solution, has several benefits. For instance, one can obtain a feasible, n5ar-optimal solution with relatively shorter amount of time. Also, if there is significant noise to the data, directly solving for the optimum may be infeasible, which could be a/voided. The trade-off is often between feasibility, computation time and accuracy of your solutions.   

A very prominent example of an approximation algorithm is _gradient descent_. For extremely high dimensional problems, directly computing global minima/maxima can be computationally challenging. Therefore gradient descent attempts to iteratively "traverse" a function and approximates the minimum/maximum. For a good intuitive reading on gradient descent, go [here](http://cs231n.github.io/optimization-1/#gd). A visualization of the technique is shown below, where the traversal of the function in iterative fashion is displayed:

<br>

![Gradient descent in action](http://charlesfranzen.com/images/gradient.png =700x)

Techniques like gradient descent can significantly reduce runtime, and can also be more conducive to parallelism. Gradient descent also allows one to control error tolerance and learning rate(the rate of traversal), yielding some control of users to more quickly obtain reasonable results. 

__Changing to a faster interface/platform__

It is often the case that simply the platform being used is inferior in speed. For example, python inherently is very inefficient in scheduling parallel tasks - for libraries such as dispy and multiprocecssing. The same can be for languages like R. As was discussed in an earlier section, Python and R often has to carry out intrepreter operations before computations can be made, due to their flexibility and ease of deployment. One may, however, choose to use a platform such as C or C++, which are much faster. Shown below is a comparison of the computation speed of languages. One may use these speed differences as a consideration for switching or sticking to a particular platform.

![Performance Comparison of Languages](https://raid6.com.au/~onlyjob/_arena/speed_close.png =1500x)

It must be noted, however, that switching is not always the best answer. C/C++ usually lacks the diversity of available libraries and packages, which can significantly slow the speed of development. Sometimes newly developed languages that offer a significant advantage - like Julia, for example - may not gain widespread use because of the lack of community support and incompatibility to already existing data science infrastructure. This inertia can often be quite the dominant force and must be an important part of considerations when selecting the platform/language.

Moreover, it may be sufficient to make languages like Python and R faster by reducing interpreter operations. Essentialy these languages are wrapper languages that sit on top of low-level languages like C, C++, and Fortran. A very good example that strips as much python operations and rely on C is the aforementioned numpy package. Numpy operations are often in C, and is usually the sensible way to tackle large data volumes and iterative processes. 

### 4. Reduce intermediate outputs by changing the sequence of operations

### Introducing queries and joins
Another approach to process optimization is not to change the subprocesses themselves, but rather attempt to shift them around in order as to reduce the sizes of intermediate outputs. This is often the key idea behind query optimization in relational databases. For those of you not familiar with relational databases, they are in essence a database that stores data in multiple sub-tables, rather than choosing to store all the data in one table. A schema of a sample relational database is shown below:

<br>
![relational database](http://www.databaseanswers.org/data_models/imdb/images/version_showing_attributes.gif)
<br>
As shown above, the entire database is organized into sub-tables, which increases the usefulness of the information. If one is only interested in parts of the data that pertains to a certain category, one only needs to lookup information in a subset of the data. This helps reduce data loads and can enhance lookup speeds.

```
### A SQL query operation that performs a lookup on one of the sub-tables

SELECT * FROM Movies
WHERE Playing_time > 200;
```
The above SQL query is a good example of a lookup operation - called a _query_. All columns in the subtable Movies is selected, with the filtering condition that only movies with the field Playing_time having a value larger than 200. Notice that this lookup operation will have taken a considerably longer amount of time had this entire table been stored in a single tabular format.

Also notice that these tables are organized by some type of unique identifiers for each row. These identifiers are often one or two more columns that form a _key_ for the respective table. In many of the tables in the given example, these columns are made obvious by the suffix _ID_. These identifiers not only allow identification of unique elements in the sub-tables, but also connect different tables together, allowing these tables to be _joined_ when a query requires that information across multiple subtables be gathered and displayed in a single tabular format. An example is shown below:

```
SELECT MovieID, Movies.Star_Ratings, Movie_Genres.Movie_Genre_Type From Movies, Movie_Genres
WHERE Movies.MovieID = Movie_Genres.MovieID 
AND Movies.Star_Ratings > 4
AND Movie_Genres.Movie_Genre_Type != "Horror";
```

The above query outputs three columns: one that is shared (and in fact "connects" the two tables) by the tables _Movies_ and _Movie_Genre_Type_, and the other two are respectively unique to one table. This query is an example of a _join_, a process of merging two tables by some join condition that determines which rows of one table match with rows of a different table, and are column-wise concatenated. 

So in this query example, the following three things occur before the query completes its lookup and displays the results:

1) Two tables are first joined using column MovieID, which both tables have. The specific join here is a __inner join__ (there are left, right, outer, and natural joins - if interested, please read into these)  

2) The resulting joined table is filtered using the Star_Ratings and Movie_Genre_Type column in a process known as __selection__  

3) All other columns except MovieID, Star_Ratings, Movie_Genre_Type is dropped, in a process known as a __projection__

### A query optimization example: optimizing joins
Joins are typically one of the costliest procedures in queries. The process usually involves searching over the second table while blocks of the first table is sequentially read in (a block nested loop join) There are many additional types of join processes, such as the hash join, sort-merge join, and index-nested loop join, that we will not necessarily elaborate on, but strongly encourage you to read up on. 

While we won't go into too much detail on the actual mechanics of how machines may perform a join, what we can talk about is how the sequence of filter, join, and column selection processes can be rearranged to reduce the overall load and time of the query. Let us take the same join example as one directly before, but slightly modify the query.

```
SELECT temp1.MovieID, temp1.Star_Ratings, temp2.Movie_Genre_Type FROM

(SELECT MovieID, Star_Ratings FROM Movies
WHERE Star_Ratings > 4) AS temp1,

(SELECT MovieID, Movie_Genre_Type FROM Movie_Genres
WHERE Movie_Genre_Type != "Horror") AS temp2

WHERE temp1.MovieID = temp2.MovieID;
```
The output of this query is the same as the query example before. It also contains an inner join, selection, and projection process. Let's see how these two queries differ by looking at the __order__ of the operations.  


1) As SQL queries are done _inside out_, that is to say inner subqueries are executed first, the tables MOovies and Movie_Genres is first filtered using the Movie_Genre_Type and Star Ratings (__Selection__)  

2) Columns MovieID,Star_Ratings for table Movies, columns MovieID, Movie_Genre_Type for table Movie_Genres is selected and all other columns dropped. (__Projection__)  

3) The inner join is done using join condition on MovieID, which both intermediate outputs of the previous steps contain. (__Inner join__)  


Notice that nothing has really changed here, except that the join is done only after the selection and projection processes, unlike the previous case, where the join occurred before the selection and projections. Selection and projection effectively _reduces_ the inputs going into the join operation. This significantly reduces the workload of the join, and in some cases, reduce the total processing time of this query by several magnitudes, if selection and projection conditions are limiting enough.

Obviously there are a whole host of other join optimizations and query optimizations in general. One should, however, be able to grasp how this type of _reordering_ of operations can reduce intermediate results and speed up operations, without necessarily having to change the natuer of the subprocesses involved.


## Server Architecture

For the purposes of understanding why frameworks like Spark can offer such significant performance improvements and how to best optimize such systems, it is helpul to know the basic structure of modern computers. Typically, computers can be split into three major components: a processor and memory pair that operate in concert, and an interface to IO devices.

### Processor

The processor takes the form of a CPU, usually with a number of seperate processing cores (each CPU in the CDS servers has 16 discrete logical processors). The CPU has a small amount of very high performance memory available to store active data (the register file), an arithmetic logic unit (ALU) that handles mathematical and logical operations, and a number of transparent (that is, invisible to programs operating in the processor) caches to store instructions and data fetched from main memory. The operation of the register file and ALU is fairly self-explanatory; the registers serve as temporary storage for values that the ALU requires or operates on. More relevant to us are the multiple levels of caches present in modern processors. 

#### Caches

Key Term: Block - some number of bytes that represents a single unit of memory; handled as discrete components for the purposes of caching.

Typically, several caches are present in a processor in order to minimize the latency involved with reading from or writing to main memory, which is often physically distant from the processor itself. To compensate, a series of progressively larger caches around the processor itself serve to provide rapid access to data stored in main memory. To understand how this works, consider a single-core processor without caching. Each assembly language instruction must be read from the appropriate location from main memory and each operation involving IO to or from main memory will require the full delay implied by the relative slowness of system RAM and the distance between main memory and the processor. Combined, these properties can greatly reduce performance. To mitigate this problem, we might add a L1 (the lowest level and smallest volume cache) cache to the processor, between it and the channels to main memory. Successive levels of larger and correspondingly slower caches may be added above the L1 cache to provide greater redundancy and further reduce the chance of a cache miss (a block of memory is not present in any cache, requiring the processor to wait for main memory IO).

#### Cache Operation

The L1 cache in this example may operate in a number of different ways, depending on the associativity policy in the hardware. In the simplest case, a direct-mapped cache, every block in main memory is mapped to a single position in the cache, which is where a block will be located if it is present in the cache. This is a many-to-one relation, as there are obviously more blocks in main memory than in the cache. 

The Cache improves processor performance by allowing recently used data to stay physically close to the processor and in high-speed SRAM memory. This means that when the processor attemtps to access a block, it gets either a cache "hit" (block present in cache) or "miss" (block not present in cache). In the event of a hit, the data in the cache allows the processor to operate immediately, instead of waiting for main memory. In the event of a miss, the block requested is brought into the cache, replacing whatever block occupies its mapped location in the cache.

Note that other replacement strategies (i.e. replace the least recently used block) are possible, but require more complex cache layouts.

Cache writing policy varies between processor implementations. Two common strategies are write-back and write-through. In a write-back cache, if the memory block to be written is hit, only the copy stored in the cache is modified, with the corresponding block in main memory being updated only when the updated copy in the cache is replaced. This method increases efficiency, but requires additional management overhead to track stale blocks (unupdated copies in main memory). Write through caches behave much more simply; when writing to a hit block, both the block in main memory and its copy in the cache are updated. This negates the benefit provided by the cache for memory writes but simplifies cache-memory interactions.

The performance increase offered by caches in general (and multi-layered caches in particular) can be demonstrated by examining the cycles per instruction (CPI) for memory operations in a processor with caching. First consider a processor with no caching. Assume that main memory requies 70 processor cycles per interaction, and 35% of all instructions require interaction with main memory. Additionally recall that instructions themselves are housed main memory, and must be read in order to be executed. We would then expect the CPI of a *single instruction* on a processor without caching to be:

```
70 [instruction load penalty] + (.35 * 70) [expected memory IO penalty] = 95. 
```
Although these numbers are not necessarily representative, this should indicate how severe a problem memory access time can be without a cache.

Now consider the same processor with only an L1 cache added, and assume that the cache has an instruction miss rate (the likelihood that an instruction fetch misses) of 2% and a data (the likelihood that an data IO operation misses) miss rate of 4%. For the L1 cache, we don't consider access times, because the cache is directly integrated into the processor and does not introduce a relevant delay when hit. The CPI for this processor is:
```
(0.02 * 70) [expected instruction load penalty with cache] + (0.35 * 0.04 * 70) [expected memory load penalty with cache] = 2.4. 
```
This is an enormous improvement over the uncached processor.

We can get additional improvements by adding successive layers of caching, but the returns in performance gain decrease quickly, as each successive layer must be larger (increasing read/write time requirements) and further away from the processor.

### 

<br>

![](https://lh6.googleusercontent.com/S1PT4HcPFaXhMgf5lUkYJJ9LAGUMHBQHeus5EiUbpZsxF3ytqE-hC0zRTp6T6kwrZw6y4y4SrsePH9mGp0UO=w1680-h919-rw)
![](https://sanjayachauwal.files.wordpress.com/2017/10/overall.gif =550x300)  
<br>

 

## Obvious Tradeoffs in Data Engineering

### Memory versus Time Tradeoff

It is frequently the case that a program's computation time requirements can be reduced at the expense of greater memory usage and vice versa. 

A very simplistic example might be a computer program that must calculate the products of some extremely large set of integer-integer pairs, where each integer is positive and less than 128. In theory, this program could be written such that it requies no in-memory data structures beyond those required to represent three integers: the multiplicand, multiplier, and product. Such a program (ignoring optimizations that might be done by the compiler) would have to calculate each product individually. Conversely, one could create a 128 by 128  two-dimensional array representing the multiplication table for all possible input values in memory when the program starts. This program would not have to rely on the processor's ability to multiply at all, as any multiplication could be performed by array lookups. Although this example may seem extreme, this type of pre-computed multiplication table is actually used in certain embedded systems. By installing a read only memory containing a multiplication table, a device manufacturer can avoid implementing a costly adder-subtracter dependent multiplication algorithm on a device with a very weak processor.

![Multiplication ROM](https://i.stack.imgur.com/TNLtQ.png =500x)

### RAM versus HDD Tradeoff

For the purposes of this subteam it is also important to understand the properties and behaviors of the hardware components that constitute a server. The performance differences between system memory (RAM) and the hard disk are particularly severe and motivate the use of Spark over MapReduce.

In terms of impact on process performance the disparity between disk and memory IO is extremely pronounced. Typically a memory operation even without caching requires only on the order of 100s of processor clock cycles (generally speaking, the time in which the CPU does a single calculation or other operation), whereas a disk operation might require up to 1000000 clock cycles to complete. This is a consequence of the fundamental physical difference between memory, which is volatile but stores information in circuit states (i.e. a bistable element's output) and a disk drive, which records binary data by magnetizing locations on magnetic film layered on a rotating metal disk. The fact that reading or writing information in a disk drive requires the disks to be physically rotated adds significant latency, which is not present for the purely circuit-based system memory.

From a performance perspective alone, it might seem that the disk drive is much less useful than RAM, but it is important to note tha a hard disk is both the only way to store information between system restarts and vastly less expensive per unit of storage. As a result, the RAM available on the master server is only 32 gigabytes, while there is nearly a terabyte of disk space. So, although it is preferable to handle all data operations in memory, it is frequently necessary to either "spill over" to disk (writing data to the disk when space in memory runs out) or stagger processing so that there is always sufficient space to manipulate data structures in memory while reading new information in from disk as the current batch is processed to avoid bottlenecking.

These concerns are extremely relevant for the Data Engineering subteam, because it is necessary to optimize high volume data processing for the hardware that is available. For example, on systems with particularly limited RAM, the performance advantage provided by Spark (which relies heavily on in-memory operations) over MapReduce (which requires significantly more disk IO) may be largely negated.

### Other Tradeoffs

Other potential trade offs in algorithm and program design are calculation accuracy (through an analytical solution) versus speed (numerical or approximation-based algorithms), storage fault tolerance (redundant copies and replication, ensuring data integrity but using more space and requiring additional time to update) versus efficiency and speed (only one copy, so quick to update but vulnerable to loss or corruption), and code efficiency vs readability (this dichotomy is a consequence of the fact that many programming languages allow computational shortcuts such as bitwise shifts to increase efficiency in special cases, at the expense of easy human interpretability).

# UNIX/Linux Guide

Please note that the following information is an extremely truncated tutorial, which we expect to provide only the most critical information required to use a Linux machine. For more comprehensive tutorials, please see the Additional Resources section below.

## File system navigation

<br>

![](http://i1.wp.com/mycodinglab.com/wp-content/uploads/2014/01/Linux-File-System-Mycodinglab.jpg) 

All paths to files in a Unix system can be expressed in terms of their relation to the root directory (denoted by ‘/’). For example, to describe a file called text.txt that was stored in a folder called documents in the root directory, we would say /documents/text.txt. Forward slashes delimit folders, in addition to indicating the root directory. You can think of root like you might the C drive top-level folder on a Windows machine.

Note that ‘..’ denotes the parent directory, and ‘.’ denotes the current directory

### Navigation commands

#### cd: Change Directory

Takes either an absolute path (from the root directory down) or a relative path (directions from the current directory), and sets the shell’s directory to that argument. Usage: 'cd DIRECTORY'

#### ls: List (directory contents)

Lists the files and folders/directories present in the current shell’s directory. Usage: 'ls'

#### pwd: Print Working Directory

Prints the absolute path of the current shell’s directory. Usage: 'pwd'

### Modification commands

#### mkdir: Make Directory

Creates a new sub-directory with a provided name in the current directory. Usage 'mkdir \[OPTION\]... DIRECTORY...''

#### mv: Move

Moves a file from one location to another. Usage: 'mv \[OPTION\]... SOURCE DEST'

#### cp: Copy

Copies a file from one locatio to another. Usage: 'cp \[OPTION\]... SOURCE DEST'

#### rm: Remove

Removes a file from the current directory, or removes a subdirectory when the -r flag is specified. Usage: 'rm \[OPTION\]... \[FILE\]...'

## Server Access

Use of the CDS servers will require you to know the following two commands, which are used to interact with remote machines.

#### ssh: Secure Shell

Provides command line access to a remote server.

To connect to the CDS master node (located at 128.84.48.178), use:
```
ssh [username]@128.84.48.178
```
where \[username\] is your subteam username. You may use an alias set in /etc/hosts in place of the IP address if you choose to set one up.


#### scp: Secure Copy

Copies a file or directory between a local and remote server. 

To copy a file called 'text.txt' in the current working directory to a folder called 'temp' in your home directory on the server, use:
```
scp test.txt [username]@128.84.48.178:temp
```

To copy a folder called 'myfolder' in the current working directory to a folder called 'temp' in your home directory on the server, use:
```
scp -r myfolder [username]@128.84.48.178:temp
```

To copy a file called 'text.txt' in the folder mydata on the remote server to a to the current working directory, use:
```
scp [username]@128.84.48.178:mydata/test.txt .
```

To copy a folder called 'mydata' in your home directory on the remote server to a to the current working directory, use:
```
scp -r [username]@128.84.48.178:mydata .
```

## Piping

Output piping is a core feature of the Bash shell that allows you to use the output of one commmand as the input for another (hence, "piping"). Accomplished using the '|' character in the following configuration:

```
[command1] | [command2]
```
In this case, the output of command1 is provided as input for command2.

This construction probably doesn't seem very useful for the small set of commands we've covered so far (for more, see the Additional Materials section's Bash command basics link), but is is a very powerful tool when used in conjunction with a larger library of available functions. For example, the [grep](https://www.gnu.org/software/grep/manual/grep.html) command allows you to find lines that contain a particular string in some set of input lines. This functionality can allow you to easily check if a file named 'text' is present in the current directory using the command:
```
ls | grep text
```

## Redirection

On a Linux system, most programs started on the command line output on two channels: STDOUT (standard output) and STDERR (standard error), both of which are displayed in the terminal when a program prints to them. To suppress or record this output (useful for programs that produce a lot of log messages, for example), you can execute the program and redirect outputs to a file for examination later.

For example, if we have a program in the current directory called 'myprogram', you could:
*   Redirect just STDOUT: 
  ```
  ./myprogram > out.txt 
  ```
*   Redirect just STDERR
  ```
  ./myprogram 2> errors.txt 
  ```
*   Redirect both STDOUT and STDERR
  ```
  ./myprogram 2>&1 log.txt 
  ```
Conversely, you also may want to redirect the contents of a file into a program that normally accepts interactive text input. If we have 'myprogram' as previously, and a file 'inputs.txt' you can do the following:
```
./myprogram < inputs.txt
```
to convert the lines contained in the file into inputs that are passed to the program.

## Vim

Vim (Vi Improved) is a command line text editor, which you will likely need in order to edit configuration files and programs remotely. To edit a file called 'text.txt' in the current directory with vim, type:

```
vim text.txt
```
Note that if the file 'text.txt' does not exist, a new file will be created with that name when you save.

Vim has two modes that are relevant here: command mode and editing mode. When vim starts, it is in command mode, which we will discuss later. To start editing the file as you normally would, with arrow-key navigation and text input, press 'a' (append) or 'i' (insert). This will switch your vim session into editing mode, and allow you to make changes normally. To access more advanced functionality, such as undo, redo, save, and exit, you will have to switch back into command mode. This can be accomplished at any time by pressing the ESC key. In command mode, vim will respond to keyboard input as instructions, rather than text to add to the file. A few useful commands are listed below.

*   'a': append (switches to editing mode)
*   'i': insert (switches to editing mode)
*   'u': undo last change
*   'CTRL+r': redo (undo last undo)
*   ':w': write file (save current state of the file)
*   ':q': exit vim (will give a warning if you have unsaved changes)
*   ':!q': exit vim, overriding unsaved changes warning
*   ':wq' or ':x': save file and exit vim

Note that when entering commands that begin with ':' you should enter a ':' character while in command mode, which will allow you to edit a text command, which is executed when you press ENTER. For example, to save and exit a vim session in editing mode, you would press ESC to enter command mode, enter ':' (SHIFT+;) to allow line input, type either 'wc' or 'x', and press ENTER.

## More Bash Commands

Some additional commands:

sort: sorts the text in the file
uniq: displays only the unique lines in the text
wc: counts the number of words in the input
more: will display the file up to the point that fits onto the terminal screen
grep: will search lines with the given input string pattern.
less: similar to more but allows more flexibility in navigating forward and backward. It also does not require the linux kernel to read the entire file, which makes the reading more efficient 
tr: transforms string input to another output. tr ‘\[a-z\]’ ‘\[A-Z\]’ capitalizes all characters,

## Additional Resources

*   [Lecture slides for CS 2043: UNIX Tools and Scripting (sadly not currently offered)](https://cs2043-sp16.github.io/schedule.html)
*   [A Bash language and scripting tutorial](http://www.bash.academy)
*   [The Bash Guide for Beginners](https://www.tldp.org/LDP/Bash-Beginners-Guide/html/)
*   [Bash command basics](https://www.unr.edu/it/research-resources/research-computing/hpc/the-grid/using-the-grid/bash-commands)
*   [Vim Cheat Sheet](https://vim.rtorr.com/)
*   [Interactive Vim tutorial](http://www.openvim.com)
*   [Comprehensive Vim tutorial](https://linuxconfig.org/vim-tutorial)



# Hadoop and MapReduce

## Apache Hadoop

Apache hadoop is a distributed storage and processing framework for high-volume datasets. Hadoop has four major components:
*   Hadoop Common: Utilities common to other Hadoop components, folded out into their own module
*   Hadoop Distributed File System (HDFS): The file framework used by Hadoop to store data on multiple nodes to enable high speed access and redundancy
*   Hadoop YARN: The scheduling engine for Hadoop processing
*   Hadoop MapReduce: Parallel processing component of Hadoop, working in concert with the HDFS; see below for more details

### Properties of Hadoop

#### The HDFS
The distributed file system is used by Hadoop to provide data redundancy and replication. Files stored in the HDFS are split into blocks and distributed among data nodes with multiple copies stored across the network to ensure data integrity even in the event of a data node failure. Blocks are usually large, so during file access more time is typically spent on data transfer between nodes than on data lookup.

The HDFS is organized into three node types: the NameNode, SecondaryNameNode, and DataNodes. The NameNode is responsible for maintaining a table of file metadata (i.e. block locations, permissions, location in the HDFS, last access, etc.), the SecondaryNameNode provides redundancy and reduces latency for the NameNode, and the DataNodes store blocks of stored files.

#### MapReduce Architecture

Split into two major paralell components: mappers and reducers, with a data exchange "shuffle" between them. A JobTracker master node monitors pending client tasks and manages a number of subsidiary TaskTracker nodes. TaskTrackers execute jobs provided by the JobTracker, return results, and manage the shuffle when the system is switching from mapping to reducing.

#### YARN Architecture

Two master nodes: ResourceManager and ApplicationMaster. The ResourceManager acts as a resource allocator and provides event handling. The ApplicationMaster manages running applications, requests resources from the ResourceManager, and tracks NodeManager workers. NodeManagers provide compute resources that are allocated by the master nodes and maintain communication with the ApplicationMaster.

For more details, please see the [Hadoop Architecture Overview](https://ercoppa.github.io/HadoopInternals/HadoopArchitectureOverview.html)

### Mappers and Reducers

#### Mappers

Each mapper is an independent process that acts on some partition of the overall dataset concurrently with other mappers that are allocated other partitions. Each mapper gives a key-value pair of unique identifier for the input data and the output value that it produces when the process completes. The behavior of mappers can be thought of as a parallelized equivalent of a functional programming map operation, which applies a function to every element a list of inputs to procuce a list of corresponding outputs. 

In IBM’s [example](https://www.ibm.com/analytics/hadoop/mapreduce), each mapper might be assigned to a single input file of city-daily temperature pairs and be responsible for reading in relevant information.

#### Reducers

Reduce processes occur after the mappers whose outputs they depend on finish and take the set of values produced by the requisite map operations and reduce them to either one value or a set of values. This process may occur iteratively, with multiple sets of reducers that further reduce the previous set's results. This process is somewhat analogous to the fold left or fold right operations in some functional languages, which recursively apply a function to the adjacent elements of a list and the previous function output until a single result is obtained.

In IBM’s example, the reduce operation takes the temperatures for each city in the different input sets and averages the temperatures by city, to produce a set of city-temperature pairs without duplicates.

### The MapReduce Shuffle

![MR Shuffle Diagram](https://i.stack.imgur.com/aIGRQ.png) 

The shuffle process occurs between completion of mapping tasks and the beginning of the reduction process. First, mappers partition and sort their result data in memory, and then write the partitions to disk. 

Reducers then read their requisite partitions from the data node disk drives (potentially on different machines), and merges/reduces them to produce an output. This output might then either be written to the hadoop file system or passed into further reducers for additional refinement.


### MapReduce Shuffle Problems

MapReduce has a number of issues that make it unsuited for large and complex operations. In particular, the process of writing and reading paritions to and from the disk, adds significant overhead and can significantly impact performance. Additionally, the process of exchanging partitions between datanodes can be expensive in terms of network IO, and may further slow down the shuffle.